# $\hspace{1.5 cm}$ **Atención: Esta Notebook debe ejecutarse usando Google Colab**

# **Práctica 4 - Visión Artificial**

**Autores:** José María García Ortiz, Levi Malest Villarreal y Ana Gil Molina

# _Transfer learning_ para reentrenar una CNN de detección de objetos propios

## Paquetes e imports necesarios

Usaremos la librería [ultralytics](https://github.com/ultralytics/ultralytics), que incorpora arquitecturas estado del arte en detección de objetos, segmentación, clasificación y estimación de _pose_, al tiempo que es muy sencilla de utilizar. Permite generar modelos reentrenados con _datasets_ propios con relativamente poco coste de entrenamiento, y exportarlos posteriormente a modelos altamente eficientes en inferencia (ONNX, TensorRT, etc.). También posibilita la interacción sencilla con otros modelos fundacionales más potentes, como SAM o RT-DETR, entre otros:

In [ ]:
!date # Para medir el tiempo de ejecución de todo el notebook

In [ ]:
!pip install ultralytics

In [ ]:
import glob
import random
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import cv2
import shutil
import ipywidgets as widgets
import locale
import subprocess
import torch
import math

from pathlib import Path
from ultralytics import YOLO
from IPython.display import display, Image
from ipywidgets import Output
from ultralytics.utils.plotting import Annotator, colors

locale.getpreferredencoding = lambda: "UTF-8" # Para evitar posible error "A UTF-8 locale is required. Got ANSI_X3.4-1968".

## Generación de un _dataset_ artificial, de tamaño controlado

Usaremos un procedimiento simple para generar un _dataset_ de prueba, con suficiente interés como para generar un problema de detección de objetos no trivial, no fácilmente resoluble con técnicas clásicas de visión, y que sin embargo se resuelve con relativa facilidad con una arquitectura CNN estado del arte en OD, YOLOv8 (2023). Reentrenaremos dicha red (mediante _transfer learning_, partiendo de unos pesos preentrenados para resolver un problema completamente diferente, en concreto el [_dataset_ COCO](https://cocodataset.org/) con nuestro propio _dataset_ del que podemos controlar tanto el número de imágenes de entrenamiento/test/validación, como el número de objetos por imagen.

### Descarga del _dataset_ de imágenes de _background_

Para dar variedad al conjunto de fondos utilizaremos el _dataset_ [VisDrone2019](https://arxiv.org/pdf/2001.06303.pdf) (en el enlace puede consultarse directamente el _paper_ en que se publicó, donde también pueden verse varias imágenes de ejemplo). El _dataset_ completo puede encontrarse en este enlace: [https://github.com/VisDrone/VisDrone-Dataset](https://github.com/VisDrone/VisDrone-Dataset). Nosotros descargaremos sólo el subconjunto de _training_ (que, con sus 6471 imágenes, es más que suficiente para nuestros experimentos):

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-train.zip
# LINK ALTERNATIVO:
# wget https://univmurcia-my.sharepoint.com/:u:/g/personal/pedroe_um_es/ERg3cY4my41Ak8M6Yml_aJgBr0ObKRt0TDwwDTAp6vGu0A?download=1 -O VisDrone2019-DET-train.zip

In [ ]:
! rm -rf VisDrone2019-DET-train             # Borramos posible descarga anterior
! echo "Descomprimiendo imágenes de fondo..."
! unzip -q VisDrone2019-DET-train.zip       # Descomprimimos imágenes
! rm -f VisDrone2019-DET-train.zip          # Borramos .zip descargado
! echo "$(ls VisDrone2019-DET-train/images/*.jpg | wc -l) imágenes de background disponibles"

Mostramos p.e. la primera imagen (con su nombre, formato y resolución):

In [ ]:
! file VisDrone2019-DET-train/images/0000002_00005_d_0000014.jpg
plt.imshow(cv2.cvtColor(cv2.imread("VisDrone2019-DET-train/images/0000002_00005_d_0000014.jpg"),cv2.COLOR_RGB2BGR));

### Descarga del _dataset_ de imágenes para _foreground_

Para dar variedad al conjunto de objetos, usaremos el dataset ALOT ("Amsterdam Library of textures"), que puede encontrarse en este enlace: [https://aloi.science.uva.nl/public_alot/](https://aloi.science.uva.nl/public_alot/). Inicialmente, jugaremos sólo con 9 de las 250 texturas disponibles (si bien en los ejercicios se propondrá utilizar diferentes combinaciones de la totalidad de texturas, disponibles en el mismo URL, cambiando `alot9classes.tgz` por `alot250classes.tgz`):

In [ ]:
! wget https://umubox.um.es/index.php/s/1SeVmppm7mi2zlp/download -O alot5clases.tgz
# LINK ALTERNATIVO:
# wget https://univmurcia-my.sharepoint.com/:u:/g/personal/pedroe_um_es/Eerpqq8iDcBOhMDyIt8_ZDwB2P6zpeWQClQTdohDccaSSQ?download=1 -O alot9classes.tgz

Usaremos, en particular, las cinco clases del _dataset_ (etiquetas '0', ..., '4'), para cada una de las cuales se dispone de 100 imágenes (en su resolución más pequeña):

In [ ]:
! rm -rf alot5clases # Borramos posible descarga anterior
! echo -n "Descomprimiendo imágenes de texturas para objetos... "
! tar xzvf alot5clases.tgz | sort > output.txt
! echo "Hecho."
! head -n5 output.txt && echo "..." `grep ".png" output.txt | wc -l` "ficheros en total ..."  && tail -n5 output.txt

Mostramos también, p.e, la primera imagen, junto con su nombre y resolución:

In [ ]:
!file filtered/alot5/10/10_c1i.png
plt.imshow(cv2.cvtColor(cv2.imread("filtered/alot5/10/10_c1i.png"),cv2.COLOR_RGB2BGR));

Mostramos ahora las primeras 10 imágenes por cada clase (en una fila por cada una de las 9 clases consideradas):


In [ ]:
def read_images(pathname):
  path = Path(pathname)
  labels = sorted([l.name for l in path.iterdir()])
  imgs = []
  for classpath in [path / l for l in labels]:
    imgsclass = [cv2.cvtColor(cv2.imread(str(filepath)),cv2.COLOR_RGB2BGR) for filepath in sorted([p for p in classpath.iterdir()])]
    imgs.append(imgsclass)

  NC = len(imgs)                                 # Número de clases.
  NICs = [len(imgsclass) for imgsclass in imgs]  # Lista de número de imágenes por clase.
  NIs = sum(NICs)                                # Número total de imágenes.
  return imgs, labels, NC, NICs, NIs

imgs, labels, NC, NICs, NIs = read_images("filtered/alot5/")
print(f" Número total de clases: {NC}\n"
      f" Nümero total de imágenes: {NIs}\n"
      f" Número de imágenes por cada clase: ({[ (i,l+'/',ncs) for (i,l,ncs) in zip(range(len(labels)),labels,NICs)]})\n"
      f" Tipo de datos: {imgs[0][0].dtype}\n"
      f" Rango: [{np.min([np.min(img) for imgsclass in imgs for img in imgsclass])},{np.max([np.max(img) for imgsclass in imgs for img in imgsclass])}]")

In [ ]:
def plot_imgs(imgs, n_imgs_per_class=5, title=""):
  assert n_imgs_per_class>=2, "n_imgs_per_class debe ser >=2 en función plot_imgs"
  rows, cols = imgs[0][0].shape[:2]
  aspect_ratio = (rows*NC) / (cols*n_imgs_per_class)
  fig, axs, = plt.subplots(NC, n_imgs_per_class, figsize=(10.0/aspect_ratio, 10.0),sharex=True, sharey=True)
  for r in range(NC):
    for c in range(n_imgs_per_class):
      axs[r,c].imshow(imgs[r][c], cmap="gray")
      axs[r,c].axis("off")
  if(title != ""):
    fig.suptitle(title, fontsize=20, y=0.925)

plot_imgs(imgs, 10)

### Generación de "objetos"

Creamos una función auxiliar, `random_object(...)`, que toma aleatoriamente una imagen de una de las clases, y coge de ella una cierta elipse, de tamaño y rotación también aleatorios (dentro de unos márgenes). La elipse tomada será un "objeto generado aleatorio" de la clase en cuestión:

### Normalizar clases"

In [ ]:
#Devuelve la posición de un conjunto de imágenes
def normalize_index(class_num):
    mapping = {10: 0, 12: 1, 30: 2, 77: 3}
    return mapping.get(class_num, 4)

In [ ]:
def random_object(imgs, labels, imgs_per_class, marginratio=0.3, minaspectratioellipse=0.5):
    def crop_non_black(image):
        # Filas y columnas donde hay al menos un píxel que no es negro:
        active_rows = np.nonzero(np.any(image, axis=1))[0]
        active_cols = np.nonzero(np.any(image, axis=0))[0]
        y_min, y_max = active_rows[0], active_rows[-1]
        x_min, x_max = active_cols[0], active_cols[-1]
        cropped_image = image[y_min:y_max+1, x_min:x_max+1]
        return cropped_image
    #cl = int(random.choice(labels))  Código anterior

    #cl = normalize_index(int(random.choice(labels))) Primer intento
    cl = random.randint(0, len(imgs) - 1)

    #ind_img = random.randint(0,NICs[cl-1]-1) Código anterior
    #ind_img = random.randint(0,NICs[cl]) Primer intento
    ind_img = random.randint(0, len(imgs[cl]) - 1)

    #image = imgs[cl-1][ind_img] Código anterior
    image = imgs[cl][ind_img]
    H,W,_ = image.shape
    margin = int(min(H,W)*marginratio)
    mask = np.zeros((H, W, 3), dtype=np.uint8)
    center = (random.randint(margin, W-margin), random.randint(margin, H-margin))
    maxaxes = min(abs(center[0]-0), abs(center[0]-W),
                  abs(center[1]-0), abs(center[1]-H))
    axesx = random.randint(maxaxes//2, maxaxes)
    axesy = int(axesx*(minaspectratioellipse+(1.0-minaspectratioellipse)*random.random()))
    axes = (axesx, axesy)
    angle = random.randint(0, 359)
    mask = cv2.ellipse(mask, center, axes, angle, 0, 360, color=(255,255,255), thickness=-1)
    ellipse_img = np.where(mask, image, np.zeros_like(image))
    # plt.imshow(ellipse_img);
    return crop_non_black(ellipse_img), cl

Mostramos diez de estos objetos de ejemplo:

In [ ]:

fig, axes = plt.subplots(2, 5, figsize=(15,5))
for i, ax in enumerate(axes.flat):
    fg_image, cl = random_object(imgs, labels, NICs)
    ax.imshow(fg_image, cmap='gray')
    ax.set_title(f"{cl}")
plt.tight_layout()
plt.show()

### Generación de fondos

Una función similar, `random_background_image(...)`, tomará una imagen aleatoria del conjunto de posibles fondos:

In [ ]:
def random_background_image(directory, minh, minw, maxh, maxw):
    archivos_jpg = glob.glob(os.path.join(directory, '') + '*.jpg')
    image_path = random.choice(archivos_jpg)
    image = cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_RGB2BGR)
    h, w, _ = image.shape
    start_x = np.random.randint(0, w - minw + 1)
    start_y = np.random.randint(0, h - minh + 1)
    end_x = start_x + np.random.randint(minw, min(w - start_x + 1, maxw))
    end_y = start_y + np.random.randint(minh, min(h - start_y + 1, maxh))
    cropped_image = image[start_y:end_y, start_x:end_x]
    return cropped_image

Mostramos aquí también otras ocho imágenes de ejemplo de estos fondos:

In [ ]:
PATH_BG_IMGS = "VisDrone2019-DET-train/images/"
fig, axes = plt.subplots(2, 4, figsize=(15,5))
for i, ax in enumerate(axes.flat):
    bg_image = random_background_image(PATH_BG_IMGS, 240, 320, 480, 640)
    ax.imshow(bg_image)
    #ax.axis('off')
plt.tight_layout()
plt.show()

### Generación de imágenes de muestra aleatorias

Generamos ahora, mediante la función `full_random_example(...)`, imágenes de muestra completas, con un número aleatorio de objetos encima de un cierto fondo, sin intentar evitar posibles solapamientos, y difuminando ligeramente los bordes de los objetos para realizar la fusión de forma menos abrupta:

In [ ]:
def full_random_example(directory_bg, minh, minw, maxh, maxw,
                        imgs_obj, labels_obj, NICs_obj,
                        max_objs):

    def add_random_object(bg_image, obj_image):
        H,W,_ = bg_image.shape
        h,w,_ = obj_image.shape
        fg_mask = (obj_image!=0)
        start_x = np.random.randint(0, W - w + 1)
        start_y = np.random.randint(0, H - h + 1)
        bg_mask = np.zeros(bg_image.shape,dtype=bool)
        bg_mask[start_y:start_y+h,start_x:start_x+w] = fg_mask
        fg_image = np.zeros_like(bg_image)
        fg_image[start_y:start_y+h,start_x:start_x+w] = obj_image
        ncx, ncy, nw, nh = (start_x+w//2)/W, (start_y+h//2)/H, w/W, h/H
        bg_mask_float = bg_mask.astype(float)
        bg_mask_float = cv2.GaussianBlur(bg_mask_float,(35,35),35.0)
        bg_mask_float[bg_mask_float<0.5] = 0.0
        return (bg_mask_float*fg_image+(1.0-bg_mask_float)*bg_image).astype("uint8"), ncx, ncy, nw, nh
    bg_img = random_background_image(directory_bg, minh, minw, maxh, maxw)
    num_objs = random.randint(1,max_objs)
    out_rects = []
    final_img = bg_img
    for i in range(num_objs):
        obj_img, cl = random_object(imgs_obj, labels_obj, NICs_obj)
        final_img, ncx, ncy, nw, nh = add_random_object(final_img, obj_img)
        out_rects.append((cl,(ncx,ncy,nw,nh)))
    return final_img, out_rects

Mostramos 8 muestras de imágenes de ejemplo generadas de esta forma:

In [ ]:
l_imgout, l_outrects = [], []
fig, axes = plt.subplots(2, 4, figsize=(15,5))
for i, ax in enumerate(axes.flat):
    imgout, outrects = full_random_example(PATH_BG_IMGS, 240, 320, 480, 640, imgs, labels, NICs, 5)
    l_imgout.append(imgout)
    l_outrects.append(outrects)
    ax.imshow(imgout)
    #ax.axis('off')
plt.tight_layout()
plt.show()

Dado que dichas muestras de imágenes con los "objetos" aleatorios han sido generadas de forma controlada, sabemos de antemano los _bounding boxes_ de dichos objetos (_"ground truth"_), que podremos usar tanto para el entrenamiento como para la validación y el posterior test. He aquí la función `draw_labeled_example(...)` para mostrar una imagen con el correspondiente etiquetado mediante _bounding boxes_:

In [ ]:
def draw_labeled_example(imgout, out_rects, COLORS):
    def draw_rect(image, ncx, ncy, nw, nh, color=(255,255,255)):
        H,W,_ = image.shape
        x, y, w, h = int(ncx * W), int(ncy * H), int(nw * W), int(nh * H)
        imgout = cv2.rectangle(image, (x-w//2, y-h//2), (x+w//2, y+h//2), color, thickness=2)
        return imgout
    imgdraw = imgout
    for (cl,(ncx, ncy, nw, nh)) in out_rects:
        imgdraw = draw_rect(imgdraw, ncx, ncy, nw, nh, COLORS[cl-1])
    return imgdraw

Y he aquí las mismas 8 imágenes anteriores, en este caso mostradas con sus correspondientes etiquetados (cada color de caja se corresponde con una clase diferente):

In [ ]:
COLORS = [((25*i+50)%256, (40*i+150)%256, (70*i+200)%256) for i in range(NC)]
fig, axes = plt.subplots(2, 4, figsize=(15,5))
for i, ax in enumerate(axes.flat):
    imgout = draw_labeled_example(l_imgout[i], l_outrects[i], COLORS)
    ax.imshow(imgout)
    #ax.axis('off')
plt.tight_layout()

# Leyenda:
patches = []
for i, color in enumerate(COLORS, start=1):
    patch = mpatches.Patch(color=[c/255 for c in color], label=f"'{i}'")
    patches.append(patch)
plt.legend(handles=patches,  bbox_to_anchor=(1.15, 0.5), loc='lower left')
plt.show()

### Generación de un dataset completo

In [ ]:
#Devuelve una lista con la posición de cada conjunto de imágenes
def replace_values(values):
    # Define the mapping for replacements
    mapping = {'10': 0, '12': 1, '30': 2, '77': 3, '83': 4}

    # Create a new list with replaced values
    new_list = [mapping.get(value, value) for value in values]

    return new_list



Generamos ahora un dataset completo, con el número deseado de imágenes de train, de validación, y de test:



In [ ]:
def generate_dataset(name,
                     directory_bg, minh, minw, maxh, maxw,
                     imgs_obj, labels_obj, NICs_obj,
                     max_objs,
                     n_imgs_train, n_imgs_val, n_imgs_test):
    # Make datasets directory, if it doesn't exist:
    try:
        os.mkdir("datasets/")
    except:
        pass
    # Remove possible old dataset with same name:
    try:
        shutil.rmtree("datasets/"+name)
    except:
        pass
    # Prepare subdirectories structure:
    os.mkdir("datasets/"+name)
    os.mkdir("datasets/"+name+"/images")
    os.mkdir("datasets/"+name+"/labels")
    os.mkdir("datasets/"+name+"/images/train")
    os.mkdir("datasets/"+name+"/images/val")
    os.mkdir("datasets/"+name+"/images/test")
    os.mkdir("datasets/"+name+"/labels/train")
    os.mkdir("datasets/"+name+"/labels/val")
    os.mkdir("datasets/"+name+"/labels/test")
    yaml_contents = \
f"""path:  ../datasets/{name} # dataset root dir
train: images/train
val:   images/val
test:  images/test
names:
  0: zero
  1: one
  2: two
  3: three
  4: four
"""
    with open("datasets/" + name + f"/{name}.yaml", 'w') as file:
        file.write(yaml_contents)
    for i in range(n_imgs_train+n_imgs_val+n_imgs_test):
        imgout, out_objs = full_random_example(directory_bg, minh, minw, maxh, maxw,
                                               imgs_obj, labels_obj, NICs_obj,
                                               max_objs)
        if i < n_imgs_train:
            basenameimgs = "datasets/" + name + "/images/train/"
            basenamelabels = "datasets/" + name + "/labels/train/"
        elif n_imgs_train <= i < n_imgs_train + n_imgs_val:
            basenameimgs = "datasets/" + name + "/images/val/"
            basenamelabels = "datasets/" + name + "/labels/val/"
        else:
            basenameimgs = "datasets/" + name + "/images/test/"
            basenamelabels = "datasets/" + name + "/labels/test/"
        cv2.imwrite(f"{basenameimgs}/img{i+1:06d}.jpg", cv2.cvtColor(imgout,cv2.COLOR_RGB2BGR))
        with open(f"{basenamelabels}/img{i+1:06d}.txt", 'w') as file:
            for (cls, (ncx, ncy, nw, nh)) in out_objs:
                file.write(f"{cls} {ncx:.4f} {ncy:.4f} {nw:.4f} {nh:.4f}" + '\n')

generate_dataset("OD_ALOT_9C_300I", PATH_BG_IMGS, 240, 320, 480, 640, imgs, replace_values(labels), NICs, 2, 300, 200, 100)

# Nota: si quisiésemos generar un dataset con, p.e., sólo objetos de una clase, y 100 imágenes de training:
# generate_dataset("OD_ALOT_1C_100I", PATH_BG_IMGS, 240, 320, 480, 640, imgs[:1], labels[:1], NICs[:1], 2, 100, 70, 50)

## Entrenamiento con el dataset generado

In [ ]:
# Comenzamos con un modelo preentrenado...
model = YOLO("yolov8n.pt")
# Y hacemos fine-tuning durante 10 épocas con el dataset (reducido) actual.
results = model.train(data="datasets/OD_ALOT_9C_300I/OD_ALOT_9C_300I.yaml", epochs=10, workers=0)

In [ ]:
print(f"Clases detectables por el modelo: {model.names}")

## Resultados del aprendizaje

Los resultados se generan en un subdirectorio de "runs/", nuevo en cada ejecución:

In [ ]:
print(f"Modelo guardado en: {results.save_dir}")

En realidad, la variable `results` devuelta por el entrenamiento del modelo tiene bastantes campos de interés que se pueden consultar:

In [ ]:
list(filter(lambda x: x.strip("__")==x, dir(results)))

In [ ]:
results.confusion_matrix.print()

Aunque una manera muy cómoda de comprobar los resultados es, directamente, inspeccionar las imágenes generadas dentro del directorio `results.saved_dir`. La siguiente celda permite inspeccionar las diferentes imágenes de salida proporcionadas (seleccionándolas en el correspondiente menú _dropdown_:

In [ ]:
directory_path = results.save_dir
image_files = [file for file in os.listdir(directory_path) if file.endswith(('.png', '.jpg', '.jpeg'))]
dropdown = widgets.Dropdown(
    options=image_files,
    description='Selecciona una imagen:',
    disabled=False)

# Crea un widget de salida para mostrar las imágenes
output_image = Output()

# Función para mostrar la imagen seleccionada
def show_image(change):
    with output_image:
        output_image.clear_output()
        display(Image(filename=os.path.join(directory_path, change['new'])))

# Llama a show_image cuando el valor del dropdown cambie
dropdown.observe(show_image, names='value')

# Muestra el dropdown y el widget de salida
display(dropdown)
display(output_image)

De particular interés son:
* La evolución de las funciones de pérdida, y las métricas durante las épocas de entrenamiento.
* La matriz de confusión resultante (normalizada y sin normalizar).
* La curva de _precision vs. recall_ (PR).
* La comparación en un _batch_ cualquiera de validación del etiquetado original frente a la salida obtenida por el modelo entrenado.

## Salvado del modelo

Tras un entrenamiento, el modelo entrenado siempre queda disponible bajo el directorio generado "runs/detect", de modo que en posteriores ejecuciones del notebook podemos cargarlo directamente y ahorrarnos el tener que entrenarlo de nuevo:

In [ ]:
saved_model_path = model.ckpt_path
print(subprocess.run(f"ls -l {saved_model_path}", shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout)

## Descarga de un modelo ya entrenado

Descargamos un modelo ya entrenado con anterioridad...

... Y lo cargamos en la variable `model`, imprimiendo algunos de sus atributos más importantes:

In [ ]:
# Cargar el modelo entrenado
model_path = f"{results.save_dir}/weights/best.pt"
#model = YOLO("runs/detect/train2/weights/best.pt")
model = YOLO(model_path)

print(f"Clases detectables por el modelo {model.model_name} (de tipo {model.task}):\n   {model.names} ")
print(f"\nOtros datos de interés del modelo:\n   {model.overrides}")
print(f"\nEstructura de capas del modelo:")
print(model.info(detailed=True,verbose=True))

## Exportación del modelo

En cualquier momento, podemos exportar el modelo en otros formatos . Dicha exportación permite su ejecución para inferencia en diferentes _backends_ (Nvidia TensorRT, torchscript, Keras, TensorFlow, ...). P.e., en este caso lo exportaremos el formato portable [ONNX](https://onnx.ai/) (Open Neural Network eXchange), que, entre otras cosas, nos permite la visualización gráfica de su  estructura de capas (algo muy interesante en este caso, ya que YOLOv8 es una arquitectura estado del arte en detección de objetos, muy optimizada en número de parámetros, pero a cambio, ¡bastante compleja en lo que a estructura se refiere!):

In [ ]:
model.export(format="onnx")

(En este punto, se recomienda descargar el archivo .onnx generado, y subirlo a la URL indicada (https://netron.app), para poder realizar la mencionada inspección gráfica del modelo).

## Inferencia

Probamos ahora con una imagen de nuestro conjunto de test.

Ultralytics nos permite realizar inferencia sobre una imagen de modo extremadamente sencillo, y siempre se puede acceder tanto a los _bounding_boxes_ de los objetos generados como a sus clases y confianzas, todo ellos en los atributos correspondiente del campo `boxes`:

In [ ]:
testimg = cv2.cvtColor(cv2.imread("./datasets/OD_ALOT_9C_300I/images/test/img000501.jpg"),cv2.COLOR_RGB2BGR)
results = model(testimg)
boxes = results[0].boxes.xyxy.cpu().tolist()
classes = results[0].boxes.cls.cpu().tolist()
print(results[0].boxes)

Mostramos la imagen de test anotada con las correspondientes _bounding boxes_ y nombres de clases, usando la cómoda clase `Annotator` para ello:

In [ ]:
annotator = Annotator(testimg, line_width=2, example=model.names[0])
if boxes is not None:
    for box, cls in zip(boxes, classes):
        annotator.box_label(box, color=colors(int(cls), True), label=model.names[int(cls)])
plt.imshow(testimg);

### Visualización de los tensores de salida del modelo

No obstante, aunque lo anterior es muy cómodo para trabajar en inferencia, sigue siendo muy instructivo poder acceder directamente a los tensores de salida del modelo durante la inferencia sobre una imagen determinada. Para ello, intervendremos en las capas finales de dicha inferencia, para obtener los mencionados tensores de salida "crudos" (_raw_), y mostrarlos gráficamente para realizar la debida interpretación.

Para ello, en primer lugar intervenimos al modelo, añadiendo un _"forward hook"_ (función de _callback_ que se ejecuta al llegar a la ejecución de una capa determinada del modelo) a las cabezas finales (de regresión de _bounding boxes_ y de confianza de las clases) del modelo. La función de callback simplemente guardará los tensores respectivos de dichas capas en variables globales, que recuperará después para devolverlas al usuario de la función `get_results_and_output_tensors`:

In [ ]:
first_output_tensor = None
second_output_tensor = None
third_output_tensor = None
fourth_output_tensor = None

def first_hook_function(module, input, output):
  global first_output_tensor
  first_output_tensor = output[0]

def second_hook_function(module, input, output):
  global second_output_tensor
  second_output_tensor = output[0]

def third_hook_function(module, input, output):
  global third_output_tensor
  third_output_tensor = output[0]

def fourth_hook_function(module, input, output):
  global fourth_output_tensor
  fourth_output_tensor = output[0]

first_last_layer = model.model.model[22].cv3[2][2]
second_last_layer = model.model.model[22].cv3[1][2]
third_last_layer = model.model.model[22].cv3[0][2]
fourth_last_layer = model.model.model[22].dfl.conv

first_last_layer._forward_hooks.clear()
first_last_layer.register_forward_hook(first_hook_function)
second_last_layer._forward_hooks.clear()
second_last_layer.register_forward_hook(second_hook_function)
third_last_layer._forward_hooks.clear()
third_last_layer.register_forward_hook(third_hook_function)
fourth_last_layer._forward_hooks.clear()
fourth_last_layer.register_forward_hook(fourth_hook_function)

print("Última capa de procesamiento (cabeza que calcula confianza de clases, escala 1):\n  ", first_last_layer)
print("   Hooks de la capa: ", list(first_last_layer._forward_hooks.items()))
print("Última capa de procesamiento (cabeza que calcula confianza de clases, escala 2):\n  ", second_last_layer)
print("   Hooks de la capa: ", list(second_last_layer._forward_hooks.items()))
print("Última capa de procesamiento (cabeza que calcula confianza de clases, escala 3):\n  ", third_last_layer)
print("   Hooks de la capa: ", list(third_last_layer._forward_hooks.items()))
print("Última capa de procesamiento (cabeza que concatena todos los bounding boxes de todas las escalas):\n  ", fourth_last_layer)
print("   Hooks de la capa: ", list(fourth_last_layer._forward_hooks.items()))

# En esta función se realiza la inferencia, y tras ella, first_output_tensor y second_output_tensor
# contendrán la salida de las capas donde registramos los respectivos hooks. Devolvemos tanto los
# resultados postprocesados como los tensores de salida al usuario de la función:
def get_results_and_output_tensors(model, img, verbose=False):
  results = model(img)
  return results, first_output_tensor, second_output_tensor, third_output_tensor, fourth_output_tensor

Comprobamos que las dimensiones de los tensores de salida cuadran con las dimensiones utilizadas en la imagen de entrada de la red:

In [ ]:
print("Resultados:",end="")
results, first_output_tensor, second_output_tensor, third_output_tensor, fourth_output_tensor = get_results_and_output_tensors(model, testimg)
print("\nDimensiones tensor confianza clases (escala 1): ", first_output_tensor.shape)
print("Dimensiones tensor confianza clases (escala 2): ", second_output_tensor.shape)
print("Dimensiones tensor confianza clases (escala 3): ", third_output_tensor.shape)
print("Dimensiones tensor bounding_boxes: ", fourth_output_tensor.shape)

(H,W,_) = testimg.shape
print(f"\nTamaño original de imagen de entrada: {H}x{W}")
NH, NW = math.ceil(H*640/max(H,W)/32)*32, math.ceil(W*640/max(H,W)/32)*32
print(f"\nTamaño reescalado para que la dimensión mayor se ajuste a la entrada de la red\n"
      f"(640 en este caso), y sea además múltiplo de tiles de 32x32 (para permitir los \n"
      f"reescalados de la pirámide de YOLO): {NH}x{NW}")
HS1, WS1 = NH//32, NW//32
HS2, WS2 = NH//16, NW//16
HS3, WS3 = int(math.ceil(NH/8)), int(math.ceil(NW/8))
print(f"\nTamaños de imágenes reescaladas a /32 (escala 1), /16 (escala 2) y /8 (escala 3), y suma de todas las posiciones\n"
      f"resultantes: {HS1}x{WS1} + {HS2}x{WS2} + {HS3}x{WS3} = {HS1*WS1+HS2*WS2+HS3*WS3}")

Los tensores de salida de las confianzas de las clases, en realidad, toman _logits_ (valores previos a un softmax), así que aplicamos la función softmax del módulo de redes neuronales de pytorch para transformarlos en tensores de probabilidad (que sumarán entonces 1.0 a lo largo del eje correspondiente a la variación por clase). Por supuesto, repetimos la operación para los tres tensores corrspondientes a las tres escalas obtenidas:

In [ ]:
scale1sm = torch.nn.functional.softmax(first_output_tensor, dim=0)
scale2sm = torch.nn.functional.softmax(second_output_tensor, dim=0)
scale3sm = torch.nn.functional.softmax(third_output_tensor, dim=0)

Dibujamos ahora, para cada una de las tres escalas, los correspondientes tensores de salida de confianza de las clases, convenientemente divididos en "rodajas" (_slices_), una por cada clase. Es muy sencillo interpretar así que celdas del tensor poseen un alto valor de probabilidad para alguna de las clases (en este ejemplo, claramente las rodajas de las clases 'two' y 'seven', en ambos casos para la escala mayor -de nuevo, la escala 1 en este caso concreto-):

In [ ]:
import math

fig, axes = plt.subplots(1, 1, figsize=(15, 5))
fig.suptitle(f"Input image", fontsize=16, y=1.02)
axes.imshow(testimg)

for k in range(3):
    # Número dinámico de filas y columnas
    num_classes = len(model.names)
    rows = math.ceil(num_classes / 5)  # Número de filas basado en el total de clases
    cols = min(5, num_classes)        # Máximo 5 columnas por fila

    fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 3))
    fig.suptitle(f"Escala {k+1}", fontsize=16, y=1.02)

    # Convertir `axes` en una lista plana de subplots
    if rows == 1:
        axes = [axes] if cols == 1 else list(axes)  # Si es una fila, convertir a lista
    else:
        axes = list(axes.flat)  # Aplanar cuando hay múltiples filas

    tensor_to_plot = (scale1sm, scale2sm, scale3sm)[k]
    for i, ax in enumerate(axes[:num_classes]):  # Iterar solo sobre las clases existentes
        im = ax.imshow(tensor_to_plot.cpu()[i, :, :].numpy(), cmap="viridis", vmin=0.0, vmax=1.0)
        ax.set_title(f"{model.names[i]}")

    # Deshabilitar subplots sobrantes
    for ax in axes[num_classes:]:
        ax.axis("off")

    fig.tight_layout()

    # Agregar una barra de color común
    cbar = fig.colorbar(im, ax=axes[:num_classes], orientation='vertical', pad=0.02)
    cbar.set_label('Escala de Color')

    fig.show()

plt.show()


Para interpretar los _bounding boxes_, correctamente divididos también por escalas, tenemos que reformatear el cuarto tensor de salida, en el que vienen agregados. Usaremos para ello las dimensiones escaladas anteriores:

In [ ]:
print(f"Dimensiones escala 1: {HS1}x{WS1}")
print(f"Dimensiones escala 2: {HS2}x{WS2}")
print(f"Dimensiones escala 3: {HS3}x{WS3}")
scale1boxes = fourth_output_tensor[0,:,HS3*WS3+HS2*WS2:].reshape(-1,HS1,WS1)
scale2boxes = fourth_output_tensor[0,:,HS3*WS3:HS3*WS3+HS2*WS2].reshape(-1,HS2,WS2)
scale3boxes = fourth_output_tensor[0,:,:HS3*WS3].reshape(-1,HS3,WS3)
print(f"Dimensiones tensor de cajas para escala 1: {scale1boxes.shape}")
print(f"Dimensiones tensor de cajas para escala 2: {scale2boxes.shape}")
print(f"Dimensiones tensor de cajas para escala 3: {scale3boxes.shape}")

En el caso de YOLOv8, cada vector de 4 elementos codifica las esquinas de una caja, siempre con valores positivos, y relativos al centro de la misma (dado por la posición `(row,col)` en el tensor). De modo que esta sencilla función `get_box_corners_from_pos` mapeará directamente dichos valores a las coordenadas de las esquinas superior izquierda (x1,y1) e inferior derecha (x2,y2) en los tensores correspondientes a cada escala:

In [ ]:
def get_box_corners_from_pos(boxes_tensor, scale, row, col):
  tx1,ty1,tx2,ty2 = boxes_tensor[:,row,col]
  x1,y1,x2,y2 = float(col-tx1),float(row-ty1),float(col+tx2),float(row+ty2)
  return x1, y1, x2, y2

Finalmente, la siguiente función muestra, para todas aquellas posiciones en los tensores con valores superiores a un determinado umbral MIN_CONFIDENCE, las cajas propuestas (posiciones en el tensor marcadas con puntos, y cajas correspondientes dibujadas en el mismo, siempre usando el color de la clase correspondiente):

In [ ]:
def plot_boxes_for_image(testimg, scale1sm, scale2sm, scale3sm, scale1boxes, scale2boxes, scale3boxes, MIN_CONFIDENCE):
  fig, axs = plt.subplots(1,4,figsize=(16,6))
  axs[0].set_title(f"Input image")
  axs[0].imshow(testimg)
  for k, (tensorclasses, tensorboxes) in enumerate(zip([scale1sm, scale2sm, scale3sm], [scale1boxes, scale2boxes, scale3boxes])):
    num_classes, rows, cols = tensorclasses.shape
    maxcl, indmaxcl = tensorclasses.max(dim=0)
    axs[k+1].imshow(maxcl.cpu().numpy(),vmin=0.0,vmax=1.0);
    axs[k+1].set_title(f"Escala {k}")
    for row in range(rows):
      for col in range(cols):
        if maxcl[row,col] > MIN_CONFIDENCE:
          x1,y1,x2,y2 = get_box_corners_from_pos(tensorboxes, 1, row, col)
          rect = mpatches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor=np.array(colors(indmaxcl[row,col], True))/255, facecolor='none')
          axs[k+1].add_patch(rect)
          rect = mpatches.Rectangle((col-0.1, row-0.1), 0.2, 0.2, linewidth=0, edgecolor='none', facecolor=np.array(colors(indmaxcl[row,col], True))/255)
          axs[k+1].add_patch(rect)

plot_boxes_for_image(testimg, scale1sm, scale2sm, scale3sm, scale1boxes, scale2boxes, scale3boxes, MIN_CONFIDENCE=0.75)

Habitualmente, los detectores de objetos implementan después una sencilla función de postprocesamiento, que suele denominarse `non_maxima_suppression`, que unifica cada _"cluster"_ de bounding_boxes en adecuadas cajas individuales. Esto es exactamente lo que implementa la librería ultralytics en su función de inferencia de alto nivel, que, como hemos visto, devuelve una sola caja por objeto en la correspondiente variable devuelta `results`.

# Apéndice: Introspección en modelos YOLOV4

En realidad, un modelo YOLOV4 hereda de un modelo Pytorch, al que simplemente añade ciertas funcionalidades adicionales:

In [ ]:
def get_class_hierarchy(obj):
    """
    Esta función recibe un objeto y retorna una lista con su jerarquía de clases,
    comenzando por su clase inmediata y siguiendo hacia arriba hasta object.
    """
    jerarquia = []
    # Obtener la clase del objeto:
    clase_actual = type(obj)

    while clase_actual:
        # Añadir la clase actual a la lista de jerarquía:
        jerarquia.append(clase_actual)
        # Si llegamos a la clase base 'object', detener el bucle:
        if clase_actual == object:
            break
        # Obtener la siguiente clase base (nota: esto solo toma la primera clase base;
        # en Python, una clase puede heredar de múltiples clases).
        clase_actual = clase_actual.__bases__[0]

    return jerarquia

print(f"Jerarquía de clases de la clase de 'model': \n {get_class_hierarchy(model)}")
print(f"Jerarquía de clases de una de las capas finales de 'model': \n {get_class_hierarchy(model.model.model[22].dfl)}")
print(f"Acceso a los nombres de todos los submódulos (capas, con relación jerárquica de inclusión entre ellas) del modelo:")
for name, module in model.named_modules():
   print(name)

In [ ]:
!date # Para medir el tiempo de ejecución de todo el notebook